In [1]:
import pandas as pd
import tensorflow as tf
from keras.utils import load_img
import cv2
tf.__version__

'2.12.0'

In [2]:
import tensorflow as tf
from keras.utils import load_img, img_to_array
from keras.applications import ResNet50
from keras.layers import GlobalMaxPooling2D
import numpy as np
from numpy.linalg import norm
import os
from keras.preprocessing import image
import pickle

In [3]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg

In [4]:
DATASET_PATH = "C:/project/fashion/data/myntradataset/"
DATA_PATH = "C:/project/fashion/data/myntradataset/"
print(os.listdir(DATA_PATH))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/project/fashion/data/myntradataset/'

In [5]:
df = pd.read_csv(DATA_PATH + "styles.csv", on_bad_lines="skip")
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/project/fashion/data/myntradataset/styles.csv'

In [6]:
%%time
import os

def get_all_filenames(directory):
    """
    Returns a set of all filenames in the given directory.
    """
    filenames = {entry.name for entry in os.scandir(directory) if entry.is_file()}
    return filenames

images = get_all_filenames(DATASET_PATH + "images/")

Wall time: 110 ms


In [7]:
def check_image_exists(image_filename):
    """
    Checks if the desired filename exists within the filenames found in the given directory.
    Returns True if the filename exists, False otherwise.
    """
    global images
    if image_filename in images:
        return image_filename
    else:
        return np.nan

df['image'] = df["id"].apply(lambda image: check_image_exists(str(image) + ".jpg"))
df = df.reset_index(drop=True)
df.head()
print(df.shape)

(44424, 11)


In [8]:
filtered_df = df[df['articleType'].isin(['Shirts', 'Trousers', 'Jeans','Track Pants', 'Tshirts', 'Socks', 
                                         'Tops', 'Sweatshirts', 'Kurtas', 'Waistcoat', 'Sarees', 'Rain Jacket', 
                                         'Dresses', 'Night suits', 'Skirts', 'Blazers', 'Kurta Sets', 'Shrug', 
                                         'Boxers', 'Dupatta', 'Bath Robe', 'Mufflers', 'Jackets', 'Trunk', 
                                         'Lounge Pants', 'Sweaters', 'Tracksuits', 'Swimwear', 'Nightdress', 
                                         'Leggings', 'Kurtis', 'Jumpsuit', 'Robe', 'Salwar and Dupatta', 
                                         'Patiala', 'Churidar', 'Lounge Tshirts', 'Lounge Shorts', 
                                         'Nehru Jackets', 'Salwar', 'Lehenga Choli', 'Clothing Set' ])]

In [1]:
df = filtered_df.dropna(subset=['image'])
print(df.shape)

NameError: name 'filtered_df' is not defined

In [10]:
def plot_figures(figures, nrows = 1, ncols=1,figsize=(8, 8)):
    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows,figsize=figsize)
    for ind,title in enumerate(figures):
        axeslist.ravel()[ind].imshow(cv2.cvtColor(figures[title], cv2.COLOR_BGR2RGB))
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout() # optional

def img_path(img,display=False):
    if display :
        return DATA_PATH+"/images/"+img
    else :
        return DATASET_PATH+"/images/"+img

def load_image(img,display=False):
    return cv2.imread(img_path(img,display))

In [11]:

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape = (224, 224, 3))
base_model.trainable = False

# Add Layer Embedding
model = tf.keras.Sequential([
    base_model,
    GlobalMaxPooling2D()
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [12]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg


In [22]:
image_paths = ['{}.jpg'.format(id) for id in df['id']]

image_paths[0:5]

['15970.jpg', '39386.jpg', '21379.jpg', '53759.jpg', '1855.jpg']

In [23]:
pickle.dump(image_paths, open('filenames.pkl', 'wb'))

In [24]:
print(len(image_paths))

19390


In [16]:
def extract_features(img_path, model):
    filepath = os.path.join(img_path)
    img = load_img(filepath, target_size=(224, 224, 3))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    pred = model.predict(img).flatten()
    return pred / np.linalg.norm(pred)

In [25]:
feature_list = []

for file in image_paths:
    feature_list.append(extract_features(file, model))

1/1 [==============================] - 0s 164ms/step


KeyboardInterrupt: 

In [ ]:
pickle.dump(feature_list, open('embeddings.pkl', 'wb'))